# 04 MS Semantic Kernel - Plugin

## Intro 

Semantic Kernel abstracts direct communication with Azure OpenAI. Plug-ins allow combining semantic functions (LLM calls) with native functions (c# code).


## Step 1 - Read Environment / Create Semantic Kernel Instance

In [5]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta1"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using Microsoft.SemanticKernel;

using DotNetEnv;

string _configurationFile = @"../01/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("AOAI_APIKEY");
string oAiEndpoint = Environment.GetEnvironmentVariable("AOAI_ENDPOINT");
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("CHATCOMPLETION_DEPLOYMENTNAME");

IKernel kernel = new KernelBuilder()
    .WithAzureChatCompletionService(
        endpoint: oAiEndpoint, 
        apiKey: oAiApiKey,
        deploymentName: chatCompletionDeploymentName)
    .Build();

Console.WriteLine($"Semantic Kernel created...");
 

Installed Packages DotNetEnv, 2.5.0 Microsoft.SemanticKernel, 1.0.0-beta1

Semantic Kernel created...


## Step 2 - Define native function

In [11]:
using System.ComponentModel;
using Microsoft.SemanticKernel;
using System.Text.Json;
using System.Net.Http;

public class NativeFunctions {

    [SKFunction, Description("Download content from url")]
    public async Task<string> DownloadContentFromUrl(string url, int maxSize = 1000) 
    {
        Console.WriteLine(url); 
        using HttpClient httpClient = new HttpClient();
        
        HttpResponseMessage httpResponseMessage = await httpClient.GetAsync(url);
        JsonDocument jsonDocument = JsonDocument.Parse(await httpResponseMessage.Content.ReadAsStringAsync());

        JsonElement jsonElement = jsonDocument.RootElement; 
        return IterateJson(jsonElement).Substring(0, maxSize); 
    }

    string IterateJson(JsonElement root) 
    {
        string article = ""; 

        if (root.ValueKind == JsonValueKind.Array)
        {
            foreach (JsonElement element in root.EnumerateArray())
            {
                IterateJson(element);
            }
        }
        else if (root.ValueKind == JsonValueKind.Object)
        {
            foreach (JsonProperty property in root.EnumerateObject())
            {
                if (property.Name == "extract"){
                    article = property.Value.GetString()??"";
                    break; 
                }
                article = IterateJson(property.Value);
            }
        }
        return article; 
    }
}

Console.WriteLine($"Inline function defined ..."); 

Inline function defined ...


## Step 3 - Import native function

In [12]:
using Microsoft.SemanticKernel.Orchestration;

//Import native function
string plugInName = "NativeFunction";
string functionName = "DownloadContentFromUrl";

NativeFunctions nativeFunctions = new NativeFunctions();
kernel.ImportFunctions(nativeFunctions, plugInName);

ISKFunction skFunction = kernel.Functions.GetFunction(plugInName, functionName);

Console.WriteLine($"Inline function {skFunction.Name} imported...");


Inline function DownloadContentFromUrl imported...


## Step 4: Define semantic inline function

In [13]:
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;


//Define semantic function inline
string skPrompt = @"Summarize the provided unstructured text in 3 easy to understand sentences. 
                    The sentences need to be short and provide the most important content of the provided text.
                    Text to summarize: {{$input}}";

OpenAIRequestSettings openAIRequestSettings = new OpenAIRequestSettings
{
    MaxTokens = 100,
    Temperature = 0.7,
    TopP = 1,
    FrequencyPenalty = 0,
    PresencePenalty = 0,
    StopSequences = new List<string> { "\n" }
};

kernel.CreateSemanticFunction(
    promptTemplate: skPrompt, 
    functionName: "SummarizeText",
    pluginName: "SemanticFunctions", 
    requestSettings: openAIRequestSettings
);

Console.WriteLine($"Semantic inline function 'SummarizeFunction' in plug-in 'SemanticFunctions' registered...");

Semantic inline function 'SummarizeFunction' in plug-in 'SemanticFunctions' registered...


## Step 5: Define semantic function from file

In [14]:
using System.IO; 

string pluginsDirectory = Path.Combine(Directory.GetCurrentDirectory(), "Plugins");
plugInName = "TextPlugin";

kernel.ImportSemanticFunctionsFromDirectory(pluginsDirectory, plugInName);

Console.WriteLine($"Semantic functions imported from directory '{pluginsDirectory}'...");

Semantic functions imported from directory 'c:\LiL_AOAI\04\Plugins'...


## Step 6: Execute functions

In [16]:
//Execute native function (Download content from URL)
ContextVariables contextVariables = new ContextVariables();
contextVariables.Add("url", "https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&titles=Super%20Bowl&explaintext=1&exsectionformat=plain");
contextVariables.Add("maxSize", "5000");

plugInName = "NativeFunction";
functionName = "DownloadContentFromUrl";

ISKFunction skFunction = kernel.Functions.GetFunction(plugInName, functionName);
KernelResult kernelResult = await kernel.RunAsync(contextVariables, skFunction);         

string wikiArticle = kernelResult.GetValue<string>() ?? "";
Console.WriteLine($"Wiki article: {wikiArticle.Substring(0,50)}...");

//Execute semantic function (Summarize downloaded article)
plugInName = "SemanticFunctions";
functionName = "SummarizeText"; 

skFunction = kernel.Functions.GetFunction(plugInName, functionName);
kernelResult = await kernel.RunAsync(wikiArticle, skFunction);

string summarization = kernelResult.GetValue<string>() ?? "";
Console.WriteLine($"Summarization: {summarization}...");

//Execute semantic function (Extract keywords from summarization)
plugInName = "TextPlugIn";
functionName = "KeyWordExtraction";

skFunction = kernel.Functions.GetFunction(plugInName, functionName);
KernelResult result = await kernel.RunAsync(summarization, skFunction); 

string keywords = result.GetValue<string>();
Console.WriteLine($"Keywords: {keywords}...");

https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&titles=Super%20Bowl&explaintext=1&exsectionformat=plain
Wiki article: The Super Bowl is the annual league championship g...
Summarization: The Super Bowl is the annual championship game of the National Football League (NFL), played on the second Sunday in February, with the winning teams awarded the Vince Lombardi Trophy. It originated from a 1966 merger agreement between the NFL and the American Football League (AFL) to compete for a championship. The event is one of the world's most-watched sports events, commanding prime advertising slots due to its high viewership, and is also the second-largest event for American food consumption after Thanksgiving...
Keywords: Super Bowl, National Football League, Vince Lombardi Trophy, advertising, food consumption...
